# AI Implementation:

In [1]:
import random
from SourceCode import MancalaGameAI
import numpy as np
random.seed(50)

In [2]:
class RandomPlayer:
    def get_move(self, state):
        return state.random_move_generator()

In [3]:
#----------------- Minimax / AlphaBeta --------------------
def minimax_decision(state, game, depth=5):
    player = game.to_move(state)

    def max_value(state, current_depth):
        # If the game is over or we've reached the depth limit, return
        if game.terminal_test(state) or current_depth == depth:
            return game.utility(state, player)

        v = -np.inf
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a), current_depth + 1))
        return v

    def min_value(state, current_depth):
        # If the game is over or we've reached the depth limit, return
        if game.terminal_test(state) or current_depth == depth:
            return game.utility(state, player)

        v = np.inf
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a), current_depth + 1))
        return v

    # Start the minimax search from the root node with depth 0
    return max(game.actions(state), key=lambda a: min_value(game.result(state, a), 1))




def alpha_beta_cutoff_search(state, game, d=5, cutoff_test=None, eval_fn=None):
    #Search game to determine best action; use alpha-beta pruning.
    #This version cuts off search and uses an evaluation function.
    player = game.to_move(state)

    # Functions used by alpha_beta
    def max_value(state, alpha, beta, depth):
        if cutoff_test(state, depth):
            return eval_fn(state)
        v = -np.inf
        for a in game.actions(state):
            v = max(v, min_value(game.result(state, a), alpha, beta, depth + 1))
            if v >= beta:
                return v
            alpha = max(alpha, v)
        return v

    def min_value(state, alpha, beta, depth):
        if cutoff_test(state, depth):
            return eval_fn(state)
        v = np.inf
        for a in game.actions(state):
            v = min(v, max_value(game.result(state, a), alpha, beta, depth + 1))
            if v <= alpha:
                return v
            beta = min(beta, v)
        return v

    # Body of alpha_beta_cutoff_search starts here:
    # The default test cuts off at depth d or at a terminal state
    cutoff_test = (cutoff_test or (lambda state, depth: depth > d or game.terminal_test(state)))
    eval_fn = eval_fn or (lambda state: game.utility(state, player))
    best_score = -np.inf
    beta = np.inf
    best_action = None
    for a in game.actions(state):
        v = min_value(game.result(state, a), best_score, beta, 1)
        if v > best_score:
            best_score = v
            best_action = a
    return best_action

In [4]:
# AI player using Minimax
class MinimaxAI:
    def __init__(self):
        self.game_adapter = MancalaGameAI()

    def get_move(self, state):
        return minimax_decision(state, self.game_adapter, depth=5)


# AI player using Alpha-Beta Pruning
class AlphaBetaAI:
    def __init__(self, depth=5):
        self.depth = depth
        self.game_adapter = MancalaGameAI()

    def get_move(self, state):
        return alpha_beta_cutoff_search(state, self.game_adapter, d=self.depth)

In [5]:
def play_game_AI(ai_player, opponent):
    game = MancalaGameAI()
    
    state = game.initial
    
    count = 0

    while not game.terminal_test(state):
        # game.display(state)
        if game.to_move(state) == 1:
            move = ai_player.get_move(state)
        else:
            move = opponent.get_move(state)
        state = game.result(state, move)
        count += 0.5

    # game.display(state)
    winner_score = game.utility(state, 1)
    if winner_score > 0:
        # print("AI wins")
        return 1, count
    elif winner_score < 0:
        # print("Opponent wins")
        return 2, count
    else:
        # print("Tie")
        return 0, count
    
    # Simulate 100 games
def sim_games_Minimax(num_games):
    ai_player = MinimaxAI()
    random_player = RandomPlayer()
    
    ai_wins = 0
    random_wins = 0
    ties = 0
    tot_turns = 0
    
    for i in range(num_games):
        result, turns = play_game_AI(ai_player, random_player)
        tot_turns += turns
        
        if result == 1:
            ai_wins += 1
        elif result == 2:
            random_wins += 1
        else:
            ties += 1
    print("Minimax AI vs Random Player\n")
    print(f"AI Wins: {ai_wins}")
    print(f"Random Player Wins: {random_wins}")
    print(f"Ties: {ties}")
    print(f"Average Turns: {tot_turns/num_games:.2f}")

def sim_games_AlphaBeta(num_games):
    ai_player = AlphaBetaAI(5)
    random_player = RandomPlayer()
    
    ai_wins = 0
    random_wins = 0
    ties = 0
    tot_turns = 0
    
    for i in range(num_games):
        result, turns = play_game_AI(ai_player, random_player)
        tot_turns += turns
        
        if result == 1:
            ai_wins += 1
        elif result == 2:
            random_wins += 1
        else:
            ties += 1
    
    print("Alpha-Beta AI vs Random Player\n")
    print(f"AI Wins: {ai_wins}")
    print(f"Random Player Wins: {random_wins}")
    print(f"Ties: {ties}")
    print(f"Average Turns: {tot_turns/num_games:.2f}")

In [6]:
# 1 game with Minimax
sim_games_Minimax(1)

Minimax AI vs Random Player

AI Wins: 1
Random Player Wins: 0
Ties: 0
Average Turns: 30.50


In [7]:
# 1 game with Alpha-Beta
sim_games_AlphaBeta(1)

Alpha-Beta AI vs Random Player

AI Wins: 1
Random Player Wins: 0
Ties: 0
Average Turns: 35.50


In [8]:
# Simulate 100 games with Minimax AI
sim_games_Minimax(100)

Minimax AI vs Random Player

AI Wins: 99
Random Player Wins: 0
Ties: 1
Average Turns: 37.65


In [9]:
# Simulate 100 games with AlphaBeta AI
sim_games_AlphaBeta(100)

Alpha-Beta AI vs Random Player

AI Wins: 98
Random Player Wins: 2
Ties: 0
Average Turns: 37.94
